# 목차
* [Chapter 3_0 개요](#chapter3_1)

## Chapter 3_0 개요 <a class="anchor" id="chapter3_0"></a>
1. 트랜스포머 모델은 NLP 분야를 크개 발전시켰다.
    - 넓은 범위의 의존성을 관리하는 데 있어서 RNN의 한계를 극복하고 입력 시퀸스를 병렬로 처리할 수 있다.

2. 트랜스포머 모델에는 세 가지 주요 유형이 있다.
    - 인코더-디코더 트랜스포머: 번역과 같은 시퀸스-투-시퀸스 작업에 사용된다.
    - 인코더 전용 트랜스포머: BERT와 같은 모델로, 주로 이해 작업에 사용된다.
    - 디코더 전용 트랜스포머: GPT 시리즈와 같은 모델로, 주로 생성 작업에 사용된다.

3. 트랜스포머 구조의 핵심 구성 요소는 다음과 같다.
    - 셀프 어텐션 메커니즘: 입력 시퀸스의 다른 위치에 있는 단어들과 얼마나 관련이 있는지 평가한다.
    - 포지셔널 인코딩: 시퀸스 내에서 단어의 순서를 나타낸다.
    - 피드포워드 신경망: 각 위치에서 독립적으로 적용되는 완전 연결 신경망이다.

4. 트랜스포머는 모든 토큰을 동시에 처리한다.
    - 토큰을 병렬 처리함에도 불구하고, 위치 인코딩을 사용해 순차적인 맥락을 유지한다.

5. 디코더 기반 트랜스포머는 아래의 그림처럼 디코더 블록이라는 층을 수직으로 여러 개 쌓아 구성된다.
    - 디코더를 훈련시키려면 입력 시퀸스와 한 토큰씩 앞으로 이동시킨 출력 시퀸스가 필요하다.

        ![Transformer](image/04-00-transformer.png)